# HyperSpectral YOLO Pipeline

## imports

In [2]:

from pathlib import Path
import pandas as pd
from ultralytics import YOLO
import os
import cv2
import matlab.engine
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
import shutil

# from google.colab import files
# from google.colab import drive

import warnings
warnings.filterwarnings("ignore")
from ultralytics import YOLO, checks
import torch
checks()

Ultralytics 8.3.59  Python-3.12.6 torch-2.6.0.dev20241216+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
Setup complete  (12 CPUs, 31.9 GB RAM, 772.3/930.9 GB disk)


In [3]:
eng = matlab.engine.start_matlab()
eng.quit()

In [ ]:


def detect_anomalies(image_path, dictionary_path='hs_dictionaries.mat', numb_pca_components =4,num_ica_components = 4, ICA_max_iterations = 1000, similarity_threshold =1, opts_lambda = 0.4, opts_max_iter = 100, lambda_ = 0.01, overlap_threshold = 0.2, min_area = 50,max_area=3200, min_anomalous_pixels = 20, spar =28):
    # Start MATLAB engine
    

    # Convert RGB to hyperspectral (rec_hs is returned directly)
    rec_hs = eng.convertToHyperspectral(image_path, dictionary_path, spar)

    # Perform anomaly detection
    bounding_boxes = eng.anomalyDetection(
        rec_hs,
        numb_pca_components,
        num_ica_components,
        ICA_max_iterations,
        similarity_threshold,
        opts_lambda,
        opts_max_iter,
        lambda_,
        overlap_threshold,
        min_area,
        max_area,
        min_anomalous_pixels
        
    )
    # Convert MATLAB bounding boxes to Python list
    bounding_boxes = [list(bbox) for bbox in bounding_boxes]

    return bounding_boxes


## running the models

### Detection using YOLOv8 only

In [ ]:

# Correct path to the trained weights

# Correct path to the trained weights
model = YOLO(r"yolov8m-drone.pt")

# Path to testing folder
test_images_dir = r"C:\Users\michael\Desktop\f_proj\output_images\test"  # Folder containing test images
results_output_dir = r"C:\Users\michael\Desktop\f_proj\YOLO_test_resultsoriginal"  # Directory to save results
os.makedirs(results_output_dir, exist_ok=True)

# Run inference on the testing folder
results = model.predict(
    source=test_images_dir,  # Path to test images
    save=True,               # Save annotated images
    save_txt=True,           # Save predictions in text format
    project=results_output_dir,  # Save results in this folder
    imgsz=640                # Image size
)

# Print summary of results
print("Summary of Detection Results:")
print(f"Number of images tested: {len(results)}")
print(f"Results saved in: {results_output_dir}")

# Display per-image metrics
for result in results:
    print(f"Image: {result.path}, Detections: {len(result.boxes)}")


### Detection using our Pipeline

In [ ]:


def compute_iou(boxA, boxB):
    # boxA and boxB: (x1, y1, x2, y2)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def is_contained(boxA, boxB):
    # Checks if boxA is at least 50% contained in boxB or vice versa.
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    areaA = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    areaB = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    if (interArea / areaA >= 0.5) or (interArea / areaB >= 0.5):
        return True
    return False

# Optional unified confidence threshold:
CONF_THRESHOLD = 0.25

# Adjustable thresholds:
YOLO_OVERLAP_THRESH = 0.0  # for discarding pipeline boxes based on overlap with full-image YOLO boxes
PIPELINE_DEDUP_THRESH = 0.5  # for deduplication in pipeline NMS

def cross_class_nms(detections, iou_thresh=0.5):
    """
    A simple cross-class NMS that keeps only the highest-confidence detection
    among overlapping boxes (IoU >= iou_thresh).
    Each detection is (class_id, confidence, x1, y1, x2, y2).
    """
    detections = sorted(detections, key=lambda d: d[1], reverse=True)
    final_dets = []
    used = [False] * len(detections)
    for i in range(len(detections)):
        if used[i]:
            continue
        final_dets.append(detections[i])
        for j in range(i + 1, len(detections)):
            if used[j]:
                continue
            iou_val = compute_iou(detections[i][2:6], detections[j][2:6])
            if iou_val >= iou_thresh:
                used[j] = True
    return final_dets

def merge_bounding_boxes(bboxes, proximity_threshold=20, max_area=30000):
    """
    Merge bounding boxes (list of (x, y, w, h)) if their centers are within proximity_threshold.
    Merge is done only if the new area <= max_area.
    Returns merged boxes as (x, y, w, h).
    """
    if not bboxes:
        return []
    merged = []
    used = [False] * len(bboxes)
    for i in range(len(bboxes)):
        if used[i]:
            continue
        x, y, w, h = bboxes[i]
        merged_box = [x, y, x + w, y + h]
        used[i] = True
        for j in range(i + 1, len(bboxes)):
            if used[j]:
                continue
            x2, y2, w2, h2 = bboxes[j]
            centerA = ((merged_box[0] + merged_box[2]) / 2, (merged_box[1] + merged_box[3]) / 2)
            centerB = (x2 + w2 / 2, y2 + h2 / 2)
            dist = ((centerA[0] - centerB[0])**2 + (centerA[1] - centerB[1])**2)**0.5
            if dist <= proximity_threshold:
                new_x1 = min(merged_box[0], x2)
                new_y1 = min(merged_box[1], y2)
                new_x2 = max(merged_box[2], x2 + w2)
                new_y2 = max(merged_box[3], y2 + h2)
                new_w = new_x2 - new_x1
                new_h = new_y2 - new_y1
                if new_w * new_h <= max_area:
                    merged_box = [new_x1, new_y1, new_x2, new_y2]
                    used[j] = True
        merged.append((merged_box[0], merged_box[1], merged_box[2]-merged_box[0], merged_box[3]-merged_box[1]))
    return merged

def deduplicate_pipeline_dets(detections, overlap_thresh=0.5):
    """
    Deduplicate detections (list of (class, conf, x1, y1, x2, y2)) by discarding duplicates
    if IoU >= overlap_thresh or one box is contained in another.
    """
    if not detections:
        return []
    detections = sorted(detections, key=lambda d: d[1], reverse=True)
    final_dets = []
    used = [False] * len(detections)
    for i in range(len(detections)):
        if used[i]:
            continue
        det_i = detections[i]
        final_dets.append(det_i)
        for j in range(i+1, len(detections)):
            if used[j]:
                continue
            if compute_iou(det_i[2:6], detections[j][2:6]) >= overlap_thresh or is_contained(det_i[2:6], detections[j][2:6]):
                used[j] = True
    return final_dets

# *** Vectorized IoU Computation for Re-adding Non-Drone Detections ***
def compute_iou_vectorized(boxes1, boxes2):
    """
    Compute the IoU matrix between two sets of boxes.
    boxes1: numpy array of shape (N, 4)
    boxes2: numpy array of shape (M, 4)
    Returns an array of shape (N, M) with IoU values.
    """
    boxes1 = np.array(boxes1)  # shape (N, 4)
    boxes2 = np.array(boxes2)  # shape (M, 4)
    
    # Compute intersection coordinates
    xA = np.maximum(boxes1[:, None, 0], boxes2[None, :, 0])
    yA = np.maximum(boxes1[:, None, 1], boxes2[None, :, 1])
    xB = np.minimum(boxes1[:, None, 2], boxes2[None, :, 2])
    yB = np.minimum(boxes1[:, None, 3], boxes2[None, :, 3])
    
    interWidth = np.maximum(0, xB - xA + 1)
    interHeight = np.maximum(0, yB - yA + 1)
    interArea = interWidth * interHeight
    
    area1 = (boxes1[:, 2] - boxes1[:, 0] + 1) * (boxes1[:, 3] - boxes1[:, 1] + 1)
    area2 = (boxes2[:, 2] - boxes2[:, 0] + 1) * (boxes2[:, 3] - boxes2[:, 1] + 1)
    
    unionArea = area1[:, None] + area2[None, :] - interArea
    iou_matrix = interArea / unionArea
    return iou_matrix

# Main processing
paddings = [200]
model = YOLO(r"yolov8m-drone.pt")
eng = matlab.engine.start_matlab()

for padding in paddings:
    test_images_dir = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\train\images"  # Folder containing test images
    results_output_dir = fr"C:\Users\michael\Desktop\f_proj\birdrone_research_data\trainresults"
    os.makedirs(results_output_dir, exist_ok=True)
    labels_output_dir = os.path.join(results_output_dir, "labels")
    os.makedirs(labels_output_dir, exist_ok=True)
    
    test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir)]
    for image_path in test_images:
        print(f"Processing: {image_path}")
        results = model.predict(source=image_path, imgsz=640, conf=CONF_THRESHOLD, save=False)
        yolo_labels = []
        detections = results[0].boxes  # YOLO detections on the full image
        original_image = cv2.imread(image_path)
        
        # *** NEW: Keep non-drone YOLO detections separately
        yolo_detections = []
        non_drone_dets = []
        if detections is not None and len(detections) > 0:
            for box in detections:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                print(f"Full-image YOLO: Class {class_id}, Conf {confidence:.2f}, BBox ({x1}, {y1}, {x2}, {y2})")
                if class_id == 0:
                    yolo_detections.append((x1, y1, x2, y2, class_id, confidence))
                    # Save with "y" prefix
                    x_center = (x1 + x2) / 2 / original_image.shape[1]
                    y_center = (y1 + y2) / 2 / original_image.shape[0]
                    bbox_width = (x2 - x1) / original_image.shape[1]
                    bbox_height = (y2 - y1) / original_image.shape[0]
                    yolo_labels.append(f"y {class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")
                else:
                    non_drone_dets.append((x1, y1, x2, y2, class_id, confidence))
            print(f"Drones (class=0) detected in {image_path} by YOLO: {len(yolo_detections)}")
        else:
            print(f"No YOLO detections in {image_path}.")
        
        # Draw YOLO drone boxes
        for (x1, y1, x2, y2, cls_id, conf) in yolo_detections:
            cv2.rectangle(original_image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
        
        # 2) Run anomaly detection pipeline
        try:
            bounding_boxes = detect_anomalies(image_path, max_area=30000, numb_pca_components=4)
            merged_boxes = merge_bounding_boxes(bounding_boxes, proximity_threshold=20, max_area=30000)
            merged_boxes = merge_bounding_boxes(merged_boxes, proximity_threshold=20, max_area=30000)
            
            pipeline_detections_global = []
            # For each merged anomaly bounding box, run YOLO on the ROI
            for bbox in merged_boxes:
                x, y, w, h = map(int, bbox)
                # Compute padded ROI (before zooming)
                x_padded = max(x - padding, 0)
                y_padded = max(y - padding, 0)
                w_padded = min(w + 2 * padding, original_image.shape[1] - x_padded)
                h_padded = min(h + 2 * padding, original_image.shape[0] - y_padded)
                
                # -------------------------------
                # ROI CENTERING ADJUSTMENT BLOCK (1.1)
                orig_center_x = x + w / 2
                orig_center_y = y + h / 2
                padded_center_x = x_padded + w_padded / 2
                padded_center_y = y_padded + h_padded / 2
                offset_x = orig_center_x - padded_center_x
                offset_y = orig_center_y - padded_center_y
                x_padded = max(int(x_padded + offset_x), 0)
                y_padded = max(int(y_padded + offset_y), 0)
                # Recalculate ROI width and height to remain within image bounds
                w_padded = min(w + 2 * padding, original_image.shape[1] - x_padded)
                h_padded = min(h + 2 * padding, original_image.shape[0] - y_padded)
                # -------------------------------
                print(f"ROI before zooming: (x={x_padded}, y={y_padded}, w={w_padded}, h={h_padded})")
                
                # -------------------------------
                # ZOOMING BLOCK
                zoom_factor = 3 if (w_padded < 200 or h_padded < 200) else 2
                if zoom_factor > 2:
                    roi = original_image[y_padded:y_padded+h_padded, x_padded:x_padded+w_padded]
                    roi_zoomed = cv2.resize(roi, (roi.shape[1]*zoom_factor, roi.shape[0]*zoom_factor))
                    roi_for_yolo = cv2.resize(roi_zoomed, (640, 640))
                    effective_w = w_padded * zoom_factor
                    effective_h = h_padded * zoom_factor
                    print(f"Zooming applied with factor {zoom_factor}.")
                else:
                    roi = original_image[y_padded:y_padded+h_padded, x_padded:x_padded+w_padded]
                    roi_for_yolo = cv2.resize(roi, (640, 640))
                    effective_w = w_padded
                    effective_h = h_padded
                # -------------------------------
                
                roi_results = model.predict(source=roi_for_yolo, conf=CONF_THRESHOLD, save=False)
                roi_detections = roi_results[0].boxes
                if roi_detections is not None and len(roi_detections) > 0:
                    print(f"Anomaly ROI from {image_path}: (x={x_padded}, y={y_padded}, w={w_padded}, h={h_padded})")
                    roi_dets_list = []
                    for roi_box in roi_detections:
                        x1_r, y1_r, x2_r, y2_r = map(int, roi_box.xyxy[0])
                        conf_r = float(roi_box.conf[0])
                        cls_r = int(roi_box.cls[0])
                        print(f"  ROI detection: Class {cls_r}, Conf {conf_r:.2f}, BBox ({x1_r}, {y1_r}, {x2_r}, {y2_r})")
                        roi_dets_list.append((cls_r, conf_r, x1_r, y1_r, x2_r, y2_r))
                    filtered_dets = cross_class_nms(roi_dets_list, iou_thresh=0.25)
                    for (cls_r, conf_r, x1_r, y1_r, x2_r, y2_r) in filtered_dets:
                        if cls_r == 0:
                            x1_abs = x1_r * effective_w // 640 + x_padded
                            x2_abs = x2_r * effective_w // 640 + x_padded
                            y1_abs = y1_r * effective_h // 640 + y_padded
                            y2_abs = y2_r * effective_h // 640 + y_padded
                            pipeline_box = (x1_abs, y1_abs, x2_abs, y2_abs)
                            discard_pipeline = False
                            for yd in yolo_detections:
                                if compute_iou(pipeline_box, yd[:4]) > YOLO_OVERLAP_THRESH or is_contained(pipeline_box, yd[:4]):
                                    discard_pipeline = True
                                    break
                            if not discard_pipeline:
                                # NEW: Aspect Ratio Filter
                                box_width = x2_abs - x1_abs
                                box_height = y2_abs - y1_abs
                                ratio = box_width / box_height if box_height > 0 else 0
                                if ratio < 0.17 or ratio > 1.52:
                                    print(f"Discarding pipeline detection due to aspect ratio {ratio:.2f}")
                                    continue
                                print(f"  -> Pipeline DRONE candidate: Conf={conf_r:.2f}, Box=({x1_abs}, {y1_abs}, {x2_abs}, {y2_abs})")
                                pipeline_detections_global.append((cls_r, conf_r, x1_abs, y1_abs, x2_abs, y2_abs))
                else:
                    print(f"No ROI detections in ROI from {image_path}.")
            
            dedup_pipeline_dets = deduplicate_pipeline_dets(pipeline_detections_global, overlap_thresh=PIPELINE_DEDUP_THRESH)
            
            for (cls_r, conf_r, x1_abs, y1_abs, x2_abs, y2_abs) in dedup_pipeline_dets:
                final_area = (x2_abs - x1_abs) * (y2_abs - y1_abs)
                if final_area > 30000:
                    print(f"Discarding pipeline detection with area {final_area} pixels (exceeds limit)")
                    continue
                print(f"  -> Final Pipeline DRONE kept: Conf={conf_r:.2f}, Box=({x1_abs}, {y1_abs}, {x2_abs}, {y2_abs})")
                x_center = (x1_abs + x2_abs) / 2 / original_image.shape[1]
                y_center = (y1_abs + y2_abs) / 2 / original_image.shape[0]
                bbox_width = (x2_abs - x1_abs) / original_image.shape[1]
                bbox_height = (y2_abs - y1_abs) / original_image.shape[0]
                yolo_labels.append(f"p {cls_r} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")
                cv2.rectangle(original_image, (x1_abs, y1_abs), (x2_abs, y2_abs), color=(0, 255, 0), thickness=2)
            
            # *** Vectorized Re-addition of Non-Drone YOLO Detections ***
            # Convert final pipeline boxes (from yolo_labels) to absolute xyxy format.
            final_pipeline_boxes = []
            for lbl in yolo_labels:
                parts = lbl.strip().split()
                # Expect either "p ..." or "y ..." or "y2 ..." format
                prefix, cls_id, xc, yc, w, h = parts
                xc, yc, w, h = map(float, (xc, yc, w, h))
                x_min = (xc - w/2) * original_image.shape[1]
                y_min = (yc - h/2) * original_image.shape[0]
                x_max = (xc + w/2) * original_image.shape[1]
                y_max = (yc + h/2) * original_image.shape[0]
                final_pipeline_boxes.append([x_min, y_min, x_max, y_max])
            final_pipeline_boxes = np.array(final_pipeline_boxes) if final_pipeline_boxes else np.empty((0,4))
            
            # Convert non_drone_dets to numpy array (only xyxy part)
            non_drone_boxes = np.array([det[:4] for det in non_drone_dets]) if non_drone_dets else np.empty((0,4))
            if non_drone_boxes.size > 0 and final_pipeline_boxes.size > 0:
                iou_matrix = compute_iou_vectorized(non_drone_boxes, final_pipeline_boxes)
                max_iou = np.max(iou_matrix, axis=1)
            elif non_drone_boxes.size > 0:
                max_iou = np.zeros(len(non_drone_dets))
            else:
                max_iou = np.array([])
            overlap_threshold = 0.5
            indices_to_readd = np.where(max_iou < overlap_threshold)[0]
            for idx in indices_to_readd:
                nx1, ny1, nx2, ny2, ncls, nconf = non_drone_dets[idx]
                print(f"Re-adding YOLO non-drone box (class={ncls}) with no significant overlap: ({nx1}, {ny1}, {nx2}, {ny2})")
                cv2.rectangle(original_image, (nx1, ny1), (nx2, ny2), color=(255, 0, 0), thickness=2)
                x_center = (nx1 + nx2) / 2 / original_image.shape[1]
                y_center = (ny1 + ny2) / 2 / original_image.shape[0]
                bbox_width = (nx2 - nx1) / original_image.shape[1]
                bbox_height = (ny2 - ny1) / original_image.shape[0]
                yolo_labels.append(f"y {ncls} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")
            
            output_path = os.path.join(results_output_dir, os.path.basename(image_path))
            cv2.imwrite(output_path, original_image)
            
            label_path = os.path.join(labels_output_dir, os.path.splitext(os.path.basename(image_path))[0] + ".txt")
            print("Final label set:", yolo_labels)
            with open(label_path, "w") as f:
                f.write("\n".join(yolo_labels))
                
        except Exception as e:
            print(f"An error occurred while processing anomalies for {image_path}: {e}")
            print("Saving YOLO-only results for this image...")
            output_path = os.path.join(results_output_dir, os.path.basename(image_path))
            cv2.imwrite(output_path, original_image)
            label_path = os.path.join(labels_output_dir, os.path.splitext(os.path.basename(image_path))[0] + ".txt")
            with open(label_path, "w") as f:
                f.write("\n".join(yolo_labels))
                
eng.quit()
print("\nPipeline completed.")


## Metric comparison

### for txt format

##### divided by drone size

In [24]:
import os
from pathlib import Path
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

def load_yolo_labels(label_folder, normalize=False, img_width=480, img_height=640):
    """
    Load YOLO-format labels from a folder. Lines can be either:
      [class_id, x_center, y_center, width, height]
    or
      [prefix, class_id, x_center, y_center, width, height]
    where prefix can be "y", "p", or "y2".
    If any coordinate > 1, we normalize by (img_width, img_height).
    
    Returns:
      dict: { image_name: list of bounding boxes }
            each bounding box is [prefix, class_id, x_center, y_center, width, height]
            (if no prefix is found, prefix=None).
    """
    labels = {}
    for label_file in Path(label_folder).glob("*.txt"):
        image_name = label_file.stem
        with open(label_file, "r") as f:
            bboxes = []
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue

                if len(parts) == 5:
                    # old style: class_id, x_center, y_center, width, height
                    prefix = None
                    class_id, x_center, y_center, width, height = map(float, parts)
                elif len(parts) == 6:
                    # new style: prefix, class_id, x_center, y_center, width, height
                    prefix = parts[0]
                    class_id, x_center, y_center, width, height = map(float, parts[1:])
                else:
                    # unexpected format
                    print(f"Skipping line (unexpected format): {line}")
                    continue

                # If something is > 1, assume absolute coords => normalize
                if any(val > 1 for val in (x_center, y_center, width, height)):
                    x_center /= img_width
                    y_center /= img_height
                    width    /= img_width
                    height   /= img_height
                    print("Normalized:", x_center, y_center, width, height)

                bboxes.append([prefix, class_id, x_center, y_center, width, height])
            labels[image_name] = bboxes
    return labels

def yolo_to_xyxy(bbox, img_width, img_height):
    """
    Convert from [prefix, class_id, x_center, y_center, w, h]
    or [class_id, x_center, y_center, w, h]
    to [x_min, y_min, x_max, y_max].
    """
    if len(bbox) == 5:
        # old: [class_id, x_center, y_center, width, height]
        class_id, x_center, y_center, width, height = bbox
    else:
        # new: [prefix, class_id, x_center, y_center, width, height]
        _, class_id, x_center, y_center, width, height = bbox
    
    x_min = (x_center - width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    x_max = (x_center + width / 2) * img_width
    y_max = (y_center + height / 2) * img_height
    return [x_min, y_min, x_max, y_max]

def calculate_iou(box1, box2):
    x_min = max(box1[0], box2[0])
    y_min = max(box1[1], box2[1])
    x_max = min(box1[2], box2[2])
    y_max = min(box1[3], box2[3])
    
    intersection = max(0, x_max - x_min) * max(0, y_max - y_min)
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union = area_box1 + area_box2 - intersection
    return intersection / union if union > 0 else 0

def categorize_images(labels, img_width, img_height):
    """
    Categorize images based on bounding box area ratio:
      <=1%, 1%-5%, >5%
    Each bounding box is either [prefix, class_id, x_c, y_c, w, h]
    or [class_id, x_c, y_c, w, h].
    """
    categories = {"<=1%": [], "1%-5%": [], ">5%": []}
    
    for image_name, bboxes in labels.items():
        for bbox in bboxes:
            if len(bbox) == 5:
                class_id, x_c, y_c, w, h = bbox
            else:
                prefix, class_id, x_c, y_c, w, h = bbox
            
            width_pixels  = w * img_width
            height_pixels = h * img_height
            area_ratio = (width_pixels * height_pixels) / (img_width * img_height)

            if area_ratio <= 0.01:
                categories["<=1%"].append(image_name)
            elif 0.01 < area_ratio <= 0.05:
                categories["1%-5%"].append(image_name)
            else:
                categories[">5%"].append(image_name)

    return categories

def evaluate_detections(ground_truths, detections, img_width, img_height, iou_threshold=0.5, category_images=None):
    """
    Evaluate detections against ground truths using precision, recall, F1 score, and mean IoU.
    Duplicated detections that differ only by prefix (e.g., 'y' vs 'y2') are treated as identical.
    """
    tp = 0
    fp = 0
    fn = 0
    ious = []

    # If we only want to evaluate a subset of images (based on category)
    if category_images is not None:
        filtered_ground_truths = {k: v for k, v in ground_truths.items() if k in category_images}
        filtered_detections   = {k: v for k, v in detections.items()   if k in category_images}
    else:
        filtered_ground_truths = ground_truths
        filtered_detections   = detections

    for image_name, gt_bboxes in filtered_ground_truths.items():
        pred_bboxes = filtered_detections.get(image_name, [])
        
        # Deduplicate predicted bboxes ignoring prefix differences
        unique_pred_bboxes = []
        seen = set()
        for pred_bbox in pred_bboxes:
            if len(pred_bbox) == 5:
                key = tuple(pred_bbox)
            else:
                # key ignores the prefix (element 0)
                key = (pred_bbox[1], pred_bbox[2], pred_bbox[3], pred_bbox[4], pred_bbox[5])
            if key not in seen:
                seen.add(key)
                unique_pred_bboxes.append(pred_bbox)

        matched_gt = set()

        for pred_bbox in unique_pred_bboxes:
            pred_box_xyxy = yolo_to_xyxy(pred_bbox, img_width, img_height)
            match_found = False

            for i, gt_bbox in enumerate(gt_bboxes):
                if i in matched_gt:
                    continue

                gt_box_xyxy = yolo_to_xyxy(gt_bbox, img_width, img_height)
                iou = calculate_iou(pred_box_xyxy, gt_box_xyxy)
                if iou >= iou_threshold:
                    tp += 1
                    matched_gt.add(i)
                    match_found = True
                    ious.append(iou)
                    break

            if not match_found:
                fp += 1

        fn += (len(gt_bboxes) - len(matched_gt))

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1        = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    mean_iou  = np.mean(ious) if ious else 0

    return {
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Mean IoU": mean_iou,
        "True Positives": tp,
        "False Positives": fp,
        "False Negatives": fn,
    }

# Example usage
if __name__ == "__main__":
    ground_truth_folder = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\train\labels"
    yolo_only_folder    = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\YOLO_train_results\predict\labels"
    hs_pipeline_folder  = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\trainresults\labels"

    img_width, img_height = 640, 640

    # Load labels (assuming they're normalized or partially normalized)
    ground_truths = load_yolo_labels(ground_truth_folder, img_width=img_width, img_height=img_height)
    yolo_only_detections = load_yolo_labels(yolo_only_folder, img_width=img_width, img_height=img_height)
    hs_pipeline_detections = load_yolo_labels(hs_pipeline_folder, normalize=True, img_width=img_width, img_height=img_height)

    # Categorize images
    categories = categorize_images(ground_truths, img_width, img_height)

    # Evaluate each category
    for category, image_list in categories.items():
        print(f"\nMetrics for category: {category} (Image Count = {len(image_list)})")

        # Evaluate YOLO-only
        yolo_only_metrics = evaluate_detections(
            ground_truths, 
            yolo_only_detections, 
            img_width, 
            img_height, 
            category_images=image_list
        )
        print("YOLO-Only Metrics:")
        for metric, value in yolo_only_metrics.items():
            if isinstance(value, float):
                print(f"  {metric}: {value:.2f}")
            else:
                print(f"  {metric}: {value}")

        # Evaluate YOLO + HS pipeline (which may include boxes with prefix 'y2')
        hs_pipeline_metrics = evaluate_detections(
            ground_truths, 
            hs_pipeline_detections, 
            img_width, 
            img_height, 
            category_images=image_list
        )
        print("YOLO + HS Pipeline Metrics:")
        for metric, value in hs_pipeline_metrics.items():
            if isinstance(value, float):
                print(f"  {metric}: {value:.2f}")
            else:
                print(f"  {metric}: {value}")


Normalized: 0.002142334375 0.002587890625 0.0001879890625 0.00047851562500000005
Normalized: 0.0025500484375 0.0020068359375 9.033281250000001e-05 7.8125e-05
Normalized: 0.002064209375 0.0020715328125 0.00012939374999999998 0.00010009687499999999
Normalized: 0.001923828125 0.000924071875 0.00027832031250000003 0.000290528125
Normalized: 0.002033690625 0.0018518062500000002 0.0001220703125 0.00055908125
Normalized: 0.002080078125 0.001291503125 0.00010253906250000001 0.00016113281249999998
Normalized: 0.0017761234375 0.003013915625 0.000134278125 0.0002270515625
Normalized: 0.0020617671875 0.0018078609374999998 7.08e-05 0.000163575
Normalized: 0.0019323734374999998 0.0010632328125 0.000134278125 0.00018310625
Normalized: 0.0021594234375 0.0016577156249999998 0.000114746875 0.0001806640625
Normalized: 0.00287475625 0.0015148921875 0.00024658125000000003 0.00033935625
Normalized: 0.0019934078125 0.0027038578125 7.568437500000001e-05 0.00019775312500000002
Normalized: 0.002053221875 0.0018

##### overall results

In [25]:
if __name__ == "__main__":
    # Set the folder paths for the labels (all in YOLO format).
    ground_truth_folder = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\train\labels"
    yolo_only_folder    = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\YOLO_train_results\predict\labels"
    hs_pipeline_folder  = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\trainresults\labels"

    # Set image dimensions (should match your data)
    img_width, img_height = 640, 640

    # Load YOLO-format labels for ground truth and detections.
    ground_truths = load_yolo_labels(ground_truth_folder, img_width=img_width, img_height=img_height)
    yolo_only_detections = load_yolo_labels(yolo_only_folder, img_width=img_width, img_height=img_height)
    hs_pipeline_detections = load_yolo_labels(hs_pipeline_folder, normalize=True, img_width=img_width, img_height=img_height)

    # Compute overall evaluation metrics without filtering by image size category.
    overall_yolo_only_metrics = evaluate_detections(
        ground_truths,
        yolo_only_detections,
        img_width,
        img_height
    )
    overall_hs_pipeline_metrics = evaluate_detections(
        ground_truths,
        hs_pipeline_detections,
        img_width,
        img_height
    )

    # Print overall results for YOLO-only detections.
    print("Overall YOLO-Only Metrics:")
    for metric, value in overall_yolo_only_metrics.items():
        if isinstance(value, float):
            print(f"  {metric}: {value:.2f}")
        else:
            print(f"  {metric}: {value}")

    # Print overall results for YOLO + HS pipeline detections.
    print("\nOverall YOLO + HS Pipeline Metrics:")
    for metric, value in overall_hs_pipeline_metrics.items():
        if isinstance(value, float):
            print(f"  {metric}: {value:.2f}")
        else:
            print(f"  {metric}: {value}")


Normalized: 0.002142334375 0.002587890625 0.0001879890625 0.00047851562500000005
Normalized: 0.0025500484375 0.0020068359375 9.033281250000001e-05 7.8125e-05
Normalized: 0.002064209375 0.0020715328125 0.00012939374999999998 0.00010009687499999999
Normalized: 0.001923828125 0.000924071875 0.00027832031250000003 0.000290528125
Normalized: 0.002033690625 0.0018518062500000002 0.0001220703125 0.00055908125
Normalized: 0.002080078125 0.001291503125 0.00010253906250000001 0.00016113281249999998
Normalized: 0.0017761234375 0.003013915625 0.000134278125 0.0002270515625
Normalized: 0.0020617671875 0.0018078609374999998 7.08e-05 0.000163575
Normalized: 0.0019323734374999998 0.0010632328125 0.000134278125 0.00018310625
Normalized: 0.0021594234375 0.0016577156249999998 0.000114746875 0.0001806640625
Normalized: 0.00287475625 0.0015148921875 0.00024658125000000003 0.00033935625
Normalized: 0.0019934078125 0.0027038578125 7.568437500000001e-05 0.00019775312500000002
Normalized: 0.002053221875 0.0018

#### results for drones only

In [26]:
import os
from pathlib import Path
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
import numpy as np

import os
from pathlib import Path

import os
from pathlib import Path

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def load_yolo_labels(label_folder, normalize=False, img_width=480, img_height=640):
    """
    Load YOLO-format labels from a folder. Expected line formats:
      - 5 tokens: [class_id, x_center, y_center, width, height]
      - 6 tokens: either [prefix, class_id, x_center, y_center, width, height] or [class_id, confidence, x_center, y_center, width, height]
      - 7 tokens: [prefix, class_id, confidence, x_center, y_center, width, height]
    If any coordinate > 1, we assume they are absolute and normalize by (img_width, img_height).
    Returns:
      dict: { image_name: list of bounding boxes }
            Each bounding box is stored as a list:
              [prefix, class_id, confidence, x_center, y_center, width, height]
            If no prefix is present, prefix is set to None.
    """
    labels = {}
    for label_file in Path(label_folder).glob("*.txt"):
        image_name = label_file.stem
        with open(label_file, "r") as f:
            bboxes = []
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue
                if len(parts) == 5:
                    # Old style: [class_id, x_center, y_center, width, height]
                    prefix = None
                    class_id, x_center, y_center, width, height = map(float, parts)
                    conf = 1.0
                elif len(parts) == 6:
                    # Could be either:
                    #   a) [prefix, class_id, x_center, y_center, width, height]
                    #   b) [class_id, confidence, x_center, y_center, width, height]
                    if not is_number(parts[0]):
                        # Treat first token as prefix
                        prefix = parts[0]
                        class_id, x_center, y_center, width, height = map(float, parts[1:])
                        conf = 1.0
                    else:
                        prefix = None
                        class_id, conf, x_center, y_center, width, height = map(float, parts)
                elif len(parts) == 7:
                    # New style: [prefix, class_id, confidence, x_center, y_center, width, height]
                    prefix = parts[0]
                    class_id, conf, x_center, y_center, width, height = map(float, parts[1:])
                else:
                    print(f"Skipping line (unexpected format): {line}")
                    continue

                # Replace prefix 'y2' with 'y'
                if prefix == "y2":
                    prefix = "y"

                # If any coordinate is greater than 1, assume absolute values and normalize them.
                if any(val > 1 for val in (x_center, y_center, width, height)):
                    x_center /= img_width
                    y_center /= img_height
                    width    /= img_width
                    height   /= img_height
                    print("Normalized:", x_center, y_center, width, height)

                bboxes.append([prefix, class_id, conf, x_center, y_center, width, height])
            labels[image_name] = bboxes
    return labels



def filter_to_drones_only(labels, prefixes=('y', 'y2', 'p', None)):
    """
    Given a dict of {image_name: [ [prefix, class_id, conf, x_center, y_center, w, h], ... ]},
    return a new dict that keeps only bounding boxes where class_id == 0
    and prefix is in the allowed list of prefixes ('y', 'y2', 'p', or None).
    """
    filtered = {}
    for image_name, bboxes in labels.items():
        drone_bboxes = []
        for bb in bboxes:
            prefix, class_id = bb[0], bb[1]
            if int(class_id) == 0 and prefix in prefixes:
                drone_bboxes.append(bb)
        if drone_bboxes:
            filtered[image_name] = drone_bboxes
    return filtered

def yolo_to_xyxy(bbox, img_width, img_height):
    """
    Convert a bounding box from YOLO format to [x_min, y_min, x_max, y_max].
    Supports three formats:
      - [class_id, x_center, y_center, width, height] (len 5)
      - [prefix, class_id, x_center, y_center, width, height] (len 6)
      - [prefix, class_id, confidence, x_center, y_center, width, height] (len 7)
    """
    if len(bbox) == 5:
        class_id, x_center, y_center, width, height = bbox
    elif len(bbox) == 6:
        # Assume format: [prefix, class_id, x_center, y_center, width, height]
        _, class_id, x_center, y_center, width, height = bbox
    elif len(bbox) == 7:
        # Format: [prefix, class_id, confidence, x_center, y_center, width, height]
        _, class_id, _, x_center, y_center, width, height = bbox
    x_min = (x_center - width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    x_max = (x_center + width / 2) * img_width
    y_max = (y_center + height / 2) * img_height
    return [x_min, y_min, x_max, y_max]

def calculate_iou(box1, box2):
    x_min = max(box1[0], box2[0])
    y_min = max(box1[1], box2[1])
    x_max = min(box1[2], box2[2])
    y_max = min(box1[3], box2[3])
    
    intersection = max(0, x_max - x_min) * max(0, y_max - y_min)
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union = area_box1 + area_box2 - intersection
    return intersection / union if union > 0 else 0

def categorize_images(labels, img_width, img_height):
    """
    Categorize images based on bounding box area ratio:
      <=1%, 1%-5%, >5%
    """
    categories = {"<=1%": [], "1%-5%": [], ">5%": []}
    
    for image_name, bboxes in labels.items():
        for bbox in bboxes:
            # bbox format: [prefix, class_id, conf, x_center, y_center, w, h]
            _, _, _, x_c, y_c, w, h = bbox
            width_pixels  = w * img_width
            height_pixels = h * img_height
            area_ratio = (width_pixels * height_pixels) / (img_width * img_height)

            if area_ratio <= 0.01:
                categories["<=1%"].append(image_name)
            elif 0.01 < area_ratio <= 0.05:
                categories["1%-5%"].append(image_name)
            else:
                categories[">5%"].append(image_name)

    return categories

def evaluate_detections(ground_truths, detections, img_width, img_height, iou_threshold=0.5, category_images=None):
    """
    Evaluate detections by computing Precision, Recall, F1-Score, Mean IoU, and counts of TP, FP, FN.
    Both ground_truths and detections should have bounding boxes in the format:
      [prefix, class_id, confidence, x_center, y_center, width, height]
    For ground truths (which generally lack a confidence value), confidence is assumed to be 1.0.
    """
    tp = 0
    fp = 0
    fn = 0
    ious = []

    if category_images is not None:
        filtered_ground_truths = {k: v for k, v in ground_truths.items() if k in category_images}
        filtered_detections   = {k: v for k, v in detections.items()   if k in category_images}
    else:
        filtered_ground_truths = ground_truths
        filtered_detections   = detections

    for image_name, gt_bboxes in filtered_ground_truths.items():
        pred_bboxes = filtered_detections.get(image_name, [])
        
        # Deduplicate predicted bboxes (naive check)
        unique_pred_bboxes = []
        for pred_bbox in pred_bboxes:
            if pred_bbox not in unique_pred_bboxes:
                unique_pred_bboxes.append(pred_bbox)

        matched_gt = set()

        for pred_bbox in unique_pred_bboxes:
            pred_box_xyxy = yolo_to_xyxy(pred_bbox, img_width, img_height)
            match_found = False

            for i, gt_bbox in enumerate(gt_bboxes):
                if i in matched_gt:
                    continue

                gt_box_xyxy = yolo_to_xyxy(gt_bbox, img_width, img_height)
                iou = calculate_iou(pred_box_xyxy, gt_box_xyxy)
                if iou >= iou_threshold:
                    tp += 1
                    matched_gt.add(i)
                    match_found = True
                    ious.append(iou)
                    break

            if not match_found:
                fp += 1

        fn += (len(gt_bboxes) - len(matched_gt))

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1        = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    mean_iou  = np.mean(ious) if ious else 0

    return {
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Mean IoU": mean_iou,
        "True Positives": tp,
        "False Positives": fp,
        "False Negatives": fn,
    }

def compute_average_precision(ground_truths, detections, img_width, img_height, iou_threshold=0.5):
    """
    Compute average precision (AP) for the detections against the ground truths.
    For each image, predictions are sorted by their confidence scores.
    A prediction is considered a true positive if its IoU with any unmatched ground truth exceeds the threshold.
    AP is then calculated using scikit-learn's average_precision_score.
    """
    all_scores = []
    all_labels = []

    for image_name, gt_bboxes in ground_truths.items():
        pred_bboxes = detections.get(image_name, [])
        # Sort predictions by confidence descending.
        sorted_preds = sorted(pred_bboxes, key=lambda b: b[2] if (len(b) >= 7 or (len(b)==6 and b[0].isalpha())) else 1.0, reverse=True)
        detected = [False] * len(gt_bboxes)

        for pred in sorted_preds:
            pred_box = yolo_to_xyxy(pred, img_width, img_height)
            # If confidence is provided, use it; otherwise default to 1.0.
            score = pred[2] if (len(pred) >= 7 or (len(pred)==6 and pred[0].isalpha())) else 1.0
            match_found = False

            for i, gt in enumerate(gt_bboxes):
                if detected[i]:
                    continue
                gt_box = yolo_to_xyxy(gt, img_width, img_height)
                iou = calculate_iou(pred_box, gt_box)
                if iou >= iou_threshold:
                    match_found = True
                    detected[i] = True
                    break

            all_scores.append(score)
            all_labels.append(1 if match_found else 0)

    if not all_labels:
        return 0
    ap = average_precision_score(np.array(all_labels), np.array(all_scores))
    return ap

# Example usage
if __name__ == "__main__":
    # Define folders for ground truth, YOLO-only predictions, and YOLO+HS pipeline predictions.
    ground_truth_folder = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\train\labels"
    yolo_only_folder    = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\YOLO_train_results\predict\labels"
    hs_pipeline_folder  = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\trainresults\labels"

    # Adjust image dimensions if necessary.
    img_width, img_height = 640, 640

    # 1) Load all bounding boxes
    ground_truths_all = load_yolo_labels(ground_truth_folder, img_width=img_width, img_height=img_height)
    yolo_only_all     = load_yolo_labels(yolo_only_folder,    img_width=img_width, img_height=img_height)
    hs_pipeline_all   = load_yolo_labels(hs_pipeline_folder,  normalize=True, 
                                         img_width=img_width, img_height=img_height)

    # 2) Filter to keep only drone detections (class_id == 0)
    ground_truths_drones = filter_to_drones_only(ground_truths_all, prefixes=('y', 'y2', None))
    yolo_only_drones     = filter_to_drones_only(yolo_only_all, prefixes=('y', 'y2', None))
    hs_pipeline_drones   = filter_to_drones_only(hs_pipeline_all, prefixes=('y', 'y2', 'p', None))

    # 3) Categorize images by the area of the drone bounding boxes.
    categories = categorize_images(ground_truths_drones, img_width, img_height)

    # 4) Evaluate detections and compute mAP for each category.
    for category, image_list in categories.items():
        print(f"\nMetrics for category: {category} (Image Count = {len(image_list)})")

        # YOLO-only metrics
        yolo_only_metrics = evaluate_detections(
            ground_truths_drones,
            yolo_only_drones,
            img_width,
            img_height,
            category_images=image_list
        )
        mAP_yolo = compute_average_precision(
            {k: v for k, v in ground_truths_drones.items() if k in image_list},
            {k: v for k, v in yolo_only_drones.items() if k in image_list},
            img_width,
            img_height
        )
        print("YOLO-Only (Drones) Metrics:")
        for metric, value in yolo_only_metrics.items():
            if isinstance(value, float):
                print(f"  {metric}: {value:.2f}")
            else:
                print(f"  {metric}: {value}")
        print(f"  mAP: {mAP_yolo:.2f}")

        # YOLO + HS Pipeline metrics
        hs_pipeline_metrics = evaluate_detections(
            ground_truths_drones,
            hs_pipeline_drones,
            img_width,
            img_height,
            category_images=image_list
        )
        mAP_pipeline = compute_average_precision(
            {k: v for k, v in ground_truths_drones.items() if k in image_list},
            {k: v for k, v in hs_pipeline_drones.items() if k in image_list},
            img_width,
            img_height
        )
        print("YOLO + HS Pipeline (Drones) Metrics:")
        for metric, value in hs_pipeline_metrics.items():
            if isinstance(value, float):
                print(f"  {metric}: {value:.2f}")
            else:
                print(f"  {metric}: {value}")
        print(f"  mAP: {mAP_pipeline:.2f}")


Normalized: 0.002142334375 0.002587890625 0.0001879890625 0.00047851562500000005
Normalized: 0.0025500484375 0.0020068359375 9.033281250000001e-05 7.8125e-05
Normalized: 0.002064209375 0.0020715328125 0.00012939374999999998 0.00010009687499999999
Normalized: 0.001923828125 0.000924071875 0.00027832031250000003 0.000290528125
Normalized: 0.002033690625 0.0018518062500000002 0.0001220703125 0.00055908125
Normalized: 0.002080078125 0.001291503125 0.00010253906250000001 0.00016113281249999998
Normalized: 0.0017761234375 0.003013915625 0.000134278125 0.0002270515625
Normalized: 0.0020617671875 0.0018078609374999998 7.08e-05 0.000163575
Normalized: 0.0019323734374999998 0.0010632328125 0.000134278125 0.00018310625
Normalized: 0.0021594234375 0.0016577156249999998 0.000114746875 0.0001806640625
Normalized: 0.00287475625 0.0015148921875 0.00024658125000000003 0.00033935625
Normalized: 0.0019934078125 0.0027038578125 7.568437500000001e-05 0.00019775312500000002
Normalized: 0.002053221875 0.0018

##### drone only overall results

In [27]:

# Define folders for ground truth and predictions (YOLO-only and YOLO+HS Pipeline).
ground_truth_folder = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\train\labels"
yolo_only_folder    = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\YOLO_train_results\predict\labels"
hs_pipeline_folder  = r"C:\Users\michael\Desktop\f_proj\birdrone_research_data\trainresults\labels"

# Set image dimensions.
img_width, img_height = 640, 640

# 1) Load all bounding boxes from YOLO-format labels.
ground_truths_all = load_yolo_labels(
    ground_truth_folder,
    img_width=img_width, 
    img_height=img_height
)
yolo_only_all = load_yolo_labels(
    yolo_only_folder,
    img_width=img_width, 
    img_height=img_height
)
hs_pipeline_all = load_yolo_labels(
    hs_pipeline_folder,
    normalize=True,
    img_width=img_width,
    img_height=img_height
)

# 2) Filter to keep only drone detections (class_id == 0). This removes non-drone (e.g., birds) boxes.
ground_truths_drones = filter_to_drones_only(ground_truths_all, prefixes=('y', 'y2', None))
yolo_only_drones     = filter_to_drones_only(yolo_only_all, prefixes=('y', 'y2', None))
hs_pipeline_drones   = filter_to_drones_only(hs_pipeline_all, prefixes=('y', 'y2', 'p', None))

# 3) Evaluate overall metrics (across all images) for YOLO-only detections.
overall_yolo_metrics = evaluate_detections(
    ground_truths_drones,
    yolo_only_drones,
    img_width,
    img_height
)
overall_yolo_mAP = compute_average_precision(
    ground_truths_drones,
    yolo_only_drones,
    img_width,
    img_height
)

print("Overall YOLO-Only (Drones) Metrics:")
for metric, value in overall_yolo_metrics.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.2f}")
    else:
        print(f"  {metric}: {value}")
print(f"  mAP: {overall_yolo_mAP:.2f}")

# 4) Evaluate overall metrics for YOLO + HS Pipeline detections.
overall_pipeline_metrics = evaluate_detections(
    ground_truths_drones,
    hs_pipeline_drones,
    img_width,
    img_height
)
overall_pipeline_mAP = compute_average_precision(
    ground_truths_drones,
    hs_pipeline_drones,
    img_width,
    img_height
)

print("\nOverall YOLO + HS Pipeline (Drones) Metrics:")
for metric, value in overall_pipeline_metrics.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.2f}")
    else:
        print(f"  {metric}: {value}")
print(f"  mAP: {overall_pipeline_mAP:.2f}")


Normalized: 0.002142334375 0.002587890625 0.0001879890625 0.00047851562500000005
Normalized: 0.0025500484375 0.0020068359375 9.033281250000001e-05 7.8125e-05
Normalized: 0.002064209375 0.0020715328125 0.00012939374999999998 0.00010009687499999999
Normalized: 0.001923828125 0.000924071875 0.00027832031250000003 0.000290528125
Normalized: 0.002033690625 0.0018518062500000002 0.0001220703125 0.00055908125
Normalized: 0.002080078125 0.001291503125 0.00010253906250000001 0.00016113281249999998
Normalized: 0.0017761234375 0.003013915625 0.000134278125 0.0002270515625
Normalized: 0.0020617671875 0.0018078609374999998 7.08e-05 0.000163575
Normalized: 0.0019323734374999998 0.0010632328125 0.000134278125 0.00018310625
Normalized: 0.0021594234375 0.0016577156249999998 0.000114746875 0.0001806640625
Normalized: 0.00287475625 0.0015148921875 0.00024658125000000003 0.00033935625
Normalized: 0.0019934078125 0.0027038578125 7.568437500000001e-05 0.00019775312500000002
Normalized: 0.002053221875 0.0018

### for XML format

##### by drone size

In [3]:
import os
from pathlib import Path
import xml.etree.ElementTree as ET
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from statsmodels.stats.proportion import proportion_confint

def load_xml_ground_truths(xml_folder, img_width, img_height, class_mapping=None):
    """
    Load ground truth annotations stored in XML files (e.g., Pascal VOC format).
    
    Each XML file is expected to contain one or more <object> elements with:
      <name> for the class (e.g., "drone")
      <bndbox> containing: <xmin>, <ymin>, <xmax>, <ymax>
    
    The bounding box is converted to YOLO format (normalized):
      x_center = (xmin + xmax) / 2 / img_width
      y_center = (ymin + ymax) / 2 / img_height
      width    = (xmax - xmin) / img_width
      height   = (ymax - ymin) / img_height
    
    Each returned bounding box is formatted as:
      [prefix, class_id, x_center, y_center, width, height]
    where prefix is set to None (as ground truth typically does not use a prefix).
    
    Args:
      xml_folder (str): Folder containing the XML annotation files.
      img_width (int): Width of the image in pixels.
      img_height (int): Height of the image in pixels.
      class_mapping (dict): Dictionary mapping object names (strings) to class_ids (ints).
                            For example: {"drone": 0}. If None, defaults to {"drone": 0}.
    
    Returns:
      dict: { image_name: list of bounding boxes } where image_name is the XML file stem.
    """
    if class_mapping is None:
        class_mapping = {"drone": 0}
    
    labels = {}
    for xml_file in Path(xml_folder).glob("*.xml"):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        image_name = xml_file.stem  # you can also get filename from XML if desired
        
        bboxes = []
        for obj in root.findall("object"):
            name = obj.find("name").text.strip()
            if name not in class_mapping:
                # Skip objects not in our mapping
                continue
            class_id = class_mapping[name]
            
            bndbox = obj.find("bndbox")
            xmin = float(bndbox.find("xmin").text)
            ymin = float(bndbox.find("ymin").text)
            xmax = float(bndbox.find("xmax").text)
            ymax = float(bndbox.find("ymax").text)
            
            # Convert to normalized YOLO format
            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height
            
            # ground truths do not have a prefix; we set it to None
            bboxes.append([None, class_id, x_center, y_center, width, height])
        
        labels[image_name] = bboxes
    return labels

def load_yolo_labels(label_folder, normalize=False, img_width=480, img_height=640):
    """
    Load YOLO-format labels from a folder. Lines can be either:
      [class_id, x_center, y_center, width, height]
    or
      [prefix, class_id, x_center, y_center, width, height]
    where prefix can be "y", "p", or "y2".
    If any coordinate > 1, we normalize by (img_width, img_height).
    
    Returns:
      dict: { image_name: list of bounding boxes }
            each bounding box is [prefix, class_id, x_center, y_center, width, height]
            (if no prefix is found, prefix=None).
    """
    labels = {}
    for label_file in Path(label_folder).glob("*.txt"):
        image_name = label_file.stem
        with open(label_file, "r") as f:
            bboxes = []
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue

                if len(parts) == 5:
                    # old style: class_id, x_center, y_center, width, height
                    prefix = None
                    class_id, x_center, y_center, width, height = map(float, parts)
                elif len(parts) == 6:
                    # new style: prefix, class_id, x_center, y_center, width, height
                    prefix = parts[0]
                    class_id, x_center, y_center, width, height = map(float, parts[1:])
                else:
                    # unexpected format
                    print(f"Skipping line (unexpected format): {line}")
                    continue

                # If any coordinate is > 1, assume absolute coords => normalize
                if any(val > 1 for val in (x_center, y_center, width, height)):
                    x_center /= img_width
                    y_center /= img_height
                    width    /= img_width
                    height   /= img_height
                    print("Normalized:", x_center, y_center, width, height)

                bboxes.append([prefix, class_id, x_center, y_center, width, height])
            labels[image_name] = bboxes
    return labels

def yolo_to_xyxy(bbox, img_width, img_height):
    """
    Convert from [prefix, class_id, x_center, y_center, w, h]
    or [class_id, x_center, y_center, w, h]
    to [x_min, y_min, x_max, y_max].
    """
    if len(bbox) == 5:
        # old: [class_id, x_center, y_center, width, height]
        class_id, x_center, y_center, width, height = bbox
    else:
        # new: [prefix, class_id, x_center, y_center, width, height]
        _, class_id, x_center, y_center, width, height = bbox
    
    x_min = (x_center - width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    x_max = (x_center + width / 2) * img_width
    y_max = (y_center + height / 2) * img_height
    return [x_min, y_min, x_max, y_max]

def calculate_iou(box1, box2):
    x_min = max(box1[0], box2[0])
    y_min = max(box1[1], box2[1])
    x_max = min(box1[2], box2[2])
    y_max = min(box1[3], box2[3])
    
    intersection = max(0, x_max - x_min) * max(0, y_max - y_min)
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union = area_box1 + area_box2 - intersection
    return intersection / union if union > 0 else 0

def categorize_images(labels, img_width, img_height):
    """
    Categorize images based on bounding box area ratio:
      <=1%, 1%-5%, >5%
    Each bounding box is either [prefix, class_id, x_c, y_c, w, h]
    or [class_id, x_c, y_c, w, h].
    """
    categories = {"<=1%": [], "1%-5%": [], ">5%": []}
    
    for image_name, bboxes in labels.items():
        for bbox in bboxes:
            if len(bbox) == 5:
                class_id, x_c, y_c, w, h = bbox
            else:
                prefix, class_id, x_c, y_c, w, h = bbox
            
            width_pixels  = w * img_width
            height_pixels = h * img_height
            area_ratio = (width_pixels * height_pixels) / (img_width * img_height)

            if area_ratio <= 0.01:
                categories["<=1%"].append(image_name)
            elif 0.01 < area_ratio <= 0.05:
                categories["1%-5%"].append(image_name)
            else:
                categories[">5%"].append(image_name)

    return categories

def evaluate_detections(ground_truths, detections, img_width, img_height, iou_threshold=0.5, category_images=None):
    """
    Evaluate detections against ground truths using precision, recall, F1 score, and mean IoU.
    Duplicated detections that differ only by prefix (e.g., 'y' vs 'y2') are treated as identical.
    """
    tp = 0
    fp = 0
    fn = 0
    ious = []

    # If we only want to evaluate a subset of images (based on category)
    if category_images is not None:
        filtered_ground_truths = {k: v for k, v in ground_truths.items() if k in category_images}
        filtered_detections   = {k: v for k, v in detections.items()   if k in category_images}
    else:
        filtered_ground_truths = ground_truths
        filtered_detections   = detections

    for image_name, gt_bboxes in filtered_ground_truths.items():
        pred_bboxes = filtered_detections.get(image_name, [])
        
        # Deduplicate predicted bboxes ignoring prefix differences
        unique_pred_bboxes = []
        seen = set()
        for pred_bbox in pred_bboxes:
            if len(pred_bbox) == 5:
                key = tuple(pred_bbox)
            else:
                # key ignores the prefix (element 0)
                key = (pred_bbox[1], pred_bbox[2], pred_bbox[3], pred_bbox[4], pred_bbox[5])
            if key not in seen:
                seen.add(key)
                unique_pred_bboxes.append(pred_bbox)

        matched_gt = set()

        for pred_bbox in unique_pred_bboxes:
            pred_box_xyxy = yolo_to_xyxy(pred_bbox, img_width, img_height)
            match_found = False

            for i, gt_bbox in enumerate(gt_bboxes):
                if i in matched_gt:
                    continue

                gt_box_xyxy = yolo_to_xyxy(gt_bbox, img_width, img_height)
                iou = calculate_iou(pred_box_xyxy, gt_box_xyxy)
                if iou >= iou_threshold:
                    tp += 1
                    matched_gt.add(i)
                    match_found = True
                    ious.append(iou)
                    break

            if not match_found:
                fp += 1

        fn += (len(gt_bboxes) - len(matched_gt))

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1        = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    mean_iou  = np.mean(ious) if ious else 0

    return {
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Mean IoU": mean_iou,
        "True Positives": tp,
        "False Positives": fp,
        "False Negatives": fn,
    }

# Example usage

if __name__ == "__main__":
    # … your folder paths and loading code …
    ground_truth_folder = r"C:\Users\michael\Desktop\f_proj\output_xml\test"
    yolo_only_folder    = r"C:\Users\michael\Desktop\f_proj\YOLO_test_resultsoriginal\predict\labels"
    hs_pipeline_folder  = r"C:\Users\michael\Desktop\f_proj\testresults\labels"
    img_width, img_height = 640, 480

    # 1) Load
    ground_truths = load_xml_ground_truths(ground_truth_folder, img_width, img_height, class_mapping={"drone": 0})
    yolo_only_detections   = load_yolo_labels(yolo_only_folder,    img_width=img_width, img_height=img_height)
    hs_pipeline_detections = load_yolo_labels(hs_pipeline_folder, normalize=True, img_width=img_width, img_height=img_height)

    # 2) Categorize by size
    categories = categorize_images(ground_truths, img_width, img_height)

    # 3) Evaluate with CIs
    for category, image_list in categories.items():
        print(f"\nMetrics for category: {category} (Image Count = {len(image_list)})")

        # YOLO-only
        yolo_metrics = evaluate_detections(
            ground_truths,
            yolo_only_detections,
            img_width,
            img_height,
            category_images=image_list
        )
        tp = yolo_metrics["True Positives"]
        fp = yolo_metrics["False Positives"]
        fn = yolo_metrics["False Negatives"]

        # 95% Wilson CIs
        prec_n = tp + fp
        rec_n  = tp + fn
        prec_low, prec_high = proportion_confint(tp, prec_n, method="wilson") if prec_n>0 else (0,0)
        rec_low,  rec_high  = proportion_confint(tp, rec_n, method="wilson") if rec_n>0 else (0,0)

        print("YOLO-Only:")
        print(f"  Precision = {yolo_metrics['Precision']:.2f} [{prec_low:.2f}, {prec_high:.2f}]")
        print(f"  Recall    = {yolo_metrics['Recall']:.2f} [{rec_low:.2f}, {rec_high:.2f}]")
        print(f"  F1-Score  = {yolo_metrics['F1-Score']:.2f}")
        print(f"  Mean IoU  = {yolo_metrics['Mean IoU']:.2f}")
        print(f"  TP = {tp}, FP = {fp}, FN = {fn}")

        # YOLO + HS pipeline
        pipe_metrics = evaluate_detections(
            ground_truths,
            hs_pipeline_detections,
            img_width,
            img_height,
            category_images=image_list
        )
        tp = pipe_metrics["True Positives"]
        fp = pipe_metrics["False Positives"]
        fn = pipe_metrics["False Negatives"]

        prec_n = tp + fp
        rec_n  = tp + fn
        prec_low, prec_high = proportion_confint(tp, prec_n, method="wilson") if prec_n>0 else (0,0)
        rec_low,  rec_high  = proportion_confint(tp, rec_n, method="wilson") if rec_n>0 else (0,0)

        print("YOLO + HS Pipeline:")
        print(f"  Precision = {pipe_metrics['Precision']:.2f} [{prec_low:.2f}, {prec_high:.2f}]")
        print(f"  Recall    = {pipe_metrics['Recall']:.2f} [{rec_low:.2f}, {rec_high:.2f}]")
        print(f"  F1-Score  = {pipe_metrics['F1-Score']:.2f}")
        print(f"  Mean IoU  = {pipe_metrics['Mean IoU']:.2f}")
        print(f"  TP = {tp}, FP = {fp}, FN = {fn}")

Normalized: 0.00115966875 0.0041688375 9.765625e-05 0.0002907979166666667
Normalized: 0.00194091875 0.0028363708333333333 0.000263671875 0.0004904520833333333
Normalized: 0.0018359375000000001 0.0022482645833333333 0.0001171875 0.0003385416666666667

Metrics for category: <=1% (Image Count = 183)
YOLO-Only:
  Precision = 0.71 [0.63, 0.78]
  Recall    = 0.54 [0.46, 0.61]
  F1-Score  = 0.61
  Mean IoU  = 0.69
  TP = 99, FP = 40, FN = 86
YOLO + HS Pipeline:
  Precision = 0.64 [0.56, 0.71]
  Recall    = 0.58 [0.51, 0.65]
  F1-Score  = 0.61
  Mean IoU  = 0.69
  TP = 107, FP = 61, FN = 78

Metrics for category: 1%-5% (Image Count = 154)
YOLO-Only:
  Precision = 0.91 [0.85, 0.95]
  Recall    = 0.71 [0.63, 0.77]
  F1-Score  = 0.80
  Mean IoU  = 0.78
  TP = 112, FP = 11, FN = 46
YOLO + HS Pipeline:
  Precision = 0.90 [0.84, 0.94]
  Recall    = 0.77 [0.70, 0.83]
  F1-Score  = 0.83
  Mean IoU  = 0.78
  TP = 122, FP = 13, FN = 36

Metrics for category: >5% (Image Count = 150)
YOLO-Only:
  Precisio

##### overall results

In [4]:
if __name__ == "__main__":
    # Set your folder paths and image dimensions
    ground_truth_folder = r"C:\Users\michael\Desktop\f_proj\output_xml\test"
    yolo_only_folder    = r"C:\Users\michael\Desktop\f_proj\YOLO_test_resultsoriginal\predict\labels"
    hs_pipeline_folder  = r"C:\Users\michael\Desktop\f_proj\testresults\labels"
    img_width, img_height = 640, 480

    # Load ground truth annotations from XML (e.g., Pascal VOC format)
    ground_truths = load_xml_ground_truths(
        ground_truth_folder,
        img_width,
        img_height,
        class_mapping={"drone": 0}
    )
    
    # Load YOLO-format detections
    yolo_only_detections = load_yolo_labels(
        yolo_only_folder, 
        img_width=img_width, 
        img_height=img_height
    )
    hs_pipeline_detections = load_yolo_labels(
        hs_pipeline_folder, 
        normalize=True, 
        img_width=img_width, 
        img_height=img_height
    )
    
    # Compute overall evaluation metrics (no category filtering)
    overall_yolo = evaluate_detections(
        ground_truths,
        yolo_only_detections,
        img_width,
        img_height
    )
    overall_pipeline = evaluate_detections(
        ground_truths,
        hs_pipeline_detections,
        img_width,
        img_height
    )
    
    # Helper to print a metric with its 95% Wilson CI
    def print_with_ci(name, tp, fp, fn):
        # precision CI
        prec_n = tp + fp
        prec_low, prec_high = (0,0)
        if prec_n > 0:
            prec_low, prec_high = proportion_confint(tp, prec_n, method="wilson")
        # recall CI
        rec_n = tp + fn
        rec_low, rec_high = (0,0)
        if rec_n > 0:
            rec_low, rec_high = proportion_confint(tp, rec_n, method="wilson")
        # point estimates
        precision = tp / prec_n if prec_n>0 else 0
        recall    = tp / rec_n if rec_n>0 else 0

        print(f"  Precision = {precision:.2f} [{prec_low:.2f}, {prec_high:.2f}]")
        print(f"  Recall    = {recall:.2f} [{rec_low:.2f}, {rec_high:.2f}]")
    
    # Print overall results for YOLO-only
    print("Overall YOLO-Only Metrics:")
    tp, fp, fn = overall_yolo["True Positives"], overall_yolo["False Positives"], overall_yolo["False Negatives"]
    print_with_ci("YOLO", tp, fp, fn)
    print(f"  F1-Score  = {overall_yolo['F1-Score']:.2f}")
    print(f"  Mean IoU  = {overall_yolo['Mean IoU']:.2f}")
    print(f"  TP = {tp}, FP = {fp}, FN = {fn}")
    
    # Print overall results for YOLO + HS pipeline
    print("\nOverall YOLO + HS Pipeline Metrics:")
    tp, fp, fn = overall_pipeline["True Positives"], overall_pipeline["False Positives"], overall_pipeline["False Negatives"]
    print_with_ci("Pipeline", tp, fp, fn)
    print(f"  F1-Score  = {overall_pipeline['F1-Score']:.2f}")
    print(f"  Mean IoU  = {overall_pipeline['Mean IoU']:.2f}")
    print(f"  TP = {tp}, FP = {fp}, FN = {fn}")

Normalized: 0.00115966875 0.0041688375 9.765625e-05 0.0002907979166666667
Normalized: 0.00194091875 0.0028363708333333333 0.000263671875 0.0004904520833333333
Normalized: 0.0018359375000000001 0.0022482645833333333 0.0001171875 0.0003385416666666667
Overall YOLO-Only Metrics:
  Precision = 0.85 [0.81, 0.88]
  Recall    = 0.69 [0.65, 0.73]
  F1-Score  = 0.76
  Mean IoU  = 0.77
  TP = 335, FP = 58, FN = 152

Overall YOLO + HS Pipeline Metrics:
  Precision = 0.80 [0.76, 0.83]
  Recall    = 0.73 [0.69, 0.77]
  F1-Score  = 0.76
  Mean IoU  = 0.77
  TP = 355, FP = 90, FN = 132


#### resultss for drones only

##### by drone size

In [15]:
def categorize_images(labels, img_width, img_height):
    """
    Categorize images based on bounding box area ratio:
      <=1%, 1%-5%, >5%
      
    Supports bounding boxes in either format:
      - [prefix, class_id, x_center, y_center, width, height] (e.g., from XML ground truths)
      - [prefix, class_id, conf, x_center, y_center, width, height] (from detections)
    """
    categories = {"<=1%": [], "1%-5%": [], ">5%": []}
    
    for image_name, bboxes in labels.items():
        for bbox in bboxes:
            if len(bbox) == 6:
                # Format from XML ground truths: [prefix, class_id, x_center, y_center, w, h]
                _, _, x_c, y_c, w, h = bbox
            elif len(bbox) == 7:
                # Format from detections: [prefix, class_id, conf, x_c, y_c, w, h]
                _, _, _, x_c, y_c, w, h = bbox
            else:
                print(f"Unexpected bbox format for image {image_name}: {bbox}")
                continue

            width_pixels  = w * img_width
            height_pixels = h * img_height
            area_ratio = (width_pixels * height_pixels) / (img_width * img_height)

            if area_ratio <= 0.01:
                categories["<=1%"].append(image_name)
            elif 0.01 < area_ratio <= 0.05:
                categories["1%-5%"].append(image_name)
            else:
                categories[">5%"].append(image_name)
                
    return categories
# Folder paths for XML ground truth and YOLO-format prediction files.
ground_truth_folder = r"C:\Users\michael\Desktop\f_proj\output_xml\test"
yolo_only_folder    = r"C:\Users\michael\Desktop\f_proj\YOLO_test_resultsoriginal\predict\labels"
hs_pipeline_folder  = r"C:\Users\michael\Desktop\f_proj\testresults\labels"

# Set image dimensions (must match annotations and detections)
img_width, img_height = 640, 480

# 1. Load ground truths from XML files
ground_truths_all = load_xml_ground_truths(
    ground_truth_folder,
    img_width,
    img_height,
    class_mapping={"drone": 0}  # Adjust if needed
)

# 2. Load YOLO-format detections (from text files)
yolo_only_all   = load_yolo_labels(yolo_only_folder, img_width=img_width, img_height=img_height)
hs_pipeline_all = load_yolo_labels(hs_pipeline_folder, normalize=True, img_width=img_width, img_height=img_height)

# 3. Filter to keep only drone detections (drone: class_id==0)
# For ground truths, the prefix is None; ensure it's allowed.
ground_truths_drones = filter_to_drones_only(ground_truths_all, prefixes=(None,))
yolo_only_drones     = filter_to_drones_only(yolo_only_all, prefixes=('y', 'y2', None))
hs_pipeline_drones   = filter_to_drones_only(hs_pipeline_all, prefixes=('y', 'y2', 'p', None))

# 4. Categorize images by the normalized area of the drone bounding boxes.
categories = categorize_images(ground_truths_drones, img_width, img_height)

# 5. Evaluate detections (and compute mAP) for each size category.
for category, image_list in categories.items():
    print(f"\nMetrics for category: {category} (Image Count = {len(image_list)})")

    # YOLO-only evaluation for images in this category.
    yolo_only_metrics = evaluate_detections(
        ground_truths_drones,
        yolo_only_drones,
        img_width,
        img_height,
        category_images=image_list
    )
    mAP_yolo = compute_average_precision(
        {k: v for k, v in ground_truths_drones.items() if k in image_list},
        {k: v for k, v in yolo_only_drones.items() if k in image_list},
        img_width,
        img_height
    )
    print("YOLO-Only (Drones) Metrics:")
    for metric, value in yolo_only_metrics.items():
        if isinstance(value, float):
            print(f"  {metric}: {value:.2f}")
        else:
            print(f"  {metric}: {value}")
    print(f"  mAP: {mAP_yolo:.2f}")

    # YOLO + HS Pipeline evaluation for images in this category.
    hs_pipeline_metrics = evaluate_detections(
        ground_truths_drones,
        hs_pipeline_drones,
        img_width,
        img_height,
        category_images=image_list
    )
    mAP_pipeline = compute_average_precision(
        {k: v for k, v in ground_truths_drones.items() if k in image_list},
        {k: v for k, v in hs_pipeline_drones.items() if k in image_list},
        img_width,
        img_height
    )
    print("YOLO + HS Pipeline (Drones) Metrics:")
    for metric, value in hs_pipeline_metrics.items():
        if isinstance(value, float):
            print(f"  {metric}: {value:.2f}")
        else:
            print(f"  {metric}: {value}")
    print(f"  mAP: {mAP_pipeline:.2f}")


Normalized: 0.00115966875 0.0041688375 9.765625e-05 0.0002907979166666667
Normalized: 0.00194091875 0.0028363708333333333 0.000263671875 0.0004904520833333333
Normalized: 0.0018359375000000001 0.0022482645833333333 0.0001171875 0.0003385416666666667

Metrics for category: <=1% (Image Count = 183)
YOLO-Only (Drones) Metrics:
  Precision: 0.73
  Recall: 0.53
  F1-Score: 0.61
  Mean IoU: 0.69
  True Positives: 98
  False Positives: 36
  False Negatives: 87
  mAP: 0.73
YOLO + HS Pipeline (Drones) Metrics:
  Precision: 0.65
  Recall: 0.57
  F1-Score: 0.61
  Mean IoU: 0.69
  True Positives: 106
  False Positives: 58
  False Negatives: 79
  mAP: 0.65

Metrics for category: 1%-5% (Image Count = 154)
YOLO-Only (Drones) Metrics:
  Precision: 0.94
  Recall: 0.66
  F1-Score: 0.78
  Mean IoU: 0.78
  True Positives: 105
  False Positives: 7
  False Negatives: 53
  mAP: 0.94
YOLO + HS Pipeline (Drones) Metrics:
  Precision: 0.91
  Recall: 0.73
  F1-Score: 0.81
  Mean IoU: 0.78
  True Positives: 115
 

##### overall results

In [12]:

# Folder paths for XML ground truth and YOLO-format predictions.
ground_truth_folder = r"C:\Users\michael\Desktop\f_proj\output_xml\test"
    # YOLO detection results remain in text format:
yolo_only_folder    = r"C:\Users\michael\Desktop\f_proj\YOLO_test_resultsoriginal\predict\labels"
hs_pipeline_folder  = r"C:\Users\michael\Desktop\f_proj\testresults\labels"

# Set image dimensions.
img_width, img_height = 640, 480

# 1. Load ground truths from XML
ground_truths_all = load_xml_ground_truths(
    ground_truth_folder,
    img_width,
    img_height,
    class_mapping={"drone": 0}
)

# 2. Load YOLO-format detections
yolo_only_all   = load_yolo_labels(yolo_only_folder, img_width=img_width, img_height=img_height)
hs_pipeline_all = load_yolo_labels(hs_pipeline_folder, normalize=True, img_width=img_width, img_height=img_height)

# 3. Filter to keep only drone detections.
# Ground truths coming from XML have prefix None.
ground_truths_drones = filter_to_drones_only(ground_truths_all, prefixes=(None,))
yolo_only_drones     = filter_to_drones_only(yolo_only_all, prefixes=('y', 'y2', None))
hs_pipeline_drones   = filter_to_drones_only(hs_pipeline_all, prefixes=('y', 'y2', 'p', None))

# 4. Overall evaluation (all images)
overall_yolo_metrics = evaluate_detections(
    ground_truths_drones,
    yolo_only_drones,
    img_width,
    img_height
)
overall_yolo_mAP = compute_average_precision(
    ground_truths_drones,
    yolo_only_drones,
    img_width,
    img_height
)

print("Overall YOLO-Only (Drones) Metrics:")
for metric, value in overall_yolo_metrics.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.2f}")
    else:
        print(f"  {metric}: {value}")
print(f"  mAP: {overall_yolo_mAP:.2f}")

overall_pipeline_metrics = evaluate_detections(
    ground_truths_drones,
    hs_pipeline_drones,
    img_width,
    img_height
)
overall_pipeline_mAP = compute_average_precision(
    ground_truths_drones,
    hs_pipeline_drones,
    img_width,
    img_height
)

print("\nOverall YOLO + HS Pipeline (Drones) Metrics:")
for metric, value in overall_pipeline_metrics.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.2f}")
    else:
        print(f"  {metric}: {value}")
print(f"  mAP: {overall_pipeline_mAP:.2f}")


Normalized: 0.00115966875 0.0041688375 9.765625e-05 0.0002907979166666667
Normalized: 0.00194091875 0.0028363708333333333 0.000263671875 0.0004904520833333333
Normalized: 0.0018359375000000001 0.0022482645833333333 0.0001171875 0.0003385416666666667
Overall YOLO-Only (Drones) Metrics:
  Precision: 0.88
  Recall: 0.67
  F1-Score: 0.76
  Mean IoU: 0.77
  True Positives: 325
  False Positives: 46
  False Negatives: 162
  mAP: 0.88

Overall YOLO + HS Pipeline (Drones) Metrics:
  Precision: 0.81
  Recall: 0.71
  F1-Score: 0.75
  Mean IoU: 0.77
  True Positives: 345
  False Positives: 82
  False Negatives: 142
  mAP: 0.81
